La première étape de notre travail a été de récupérer les diverses données dont nous avions besoin afin d'essayer d'estimer l'impact des influenceurs de Youtube mais aussi des prix sur les livres les plus lus.

PARTIE 1: Récupérer des données sur les livres les plus lus, de différente manière

Récupération de l'API des bibliothèques de Paris afin d'estimer les cent livres les plus empruntés

Tout d'abord, formulons notre requête à l'API. On met comme limite 100 ouvrages car il s'agit de la limite de collecte possible au sein de l'API

In [1]:
import requests

response = requests.get("https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/les-1000-titres-les-plus-reserves-dans-les-bibliotheques-de-pret/records?limit=100")

# Afficher le type de contenu et le texte brut de la réponse
print("Type de contenu :", response.headers.get("Content-Type"))

Type de contenu : application/json; charset=utf-8


Une fois l'API récupérée, on observe la structure du jeu de données pour savoir comment s'appelle la variable 'résultat' ( ici 'results') à appeler afin de créer notre dataframe panda.
On observe que tous les rangs ne sont pas dans la base. Notre hypothèse optimiste est qu'il y a peut-être des types d'ouvrage non classés dans cette base ( et ne nous intéressant pas puisque nous nous intéressons seulement aux livres, dont la catégorie existe dans la base). C'est une hypothèse qui nous semble probable étant donné que nous avons par la suite créé nous-même des trous dans le classement en enlevant les DVDs de ce dernier. Cependant, nous ne pouvons pas en être sûrs.

In [3]:
wb_bibli = response.json()  # Utilisation de .json() sur l'objet réponse
print("Structure du JSON :")
print(wb_bibli)

Structure du JSON :
{'total_count': 1000, 'results': [{'rang': 12, 'type_de_document': 'Livre adulte', 'reservations': 522.0, 'titre': "Réinventer l'amour : comment le patriarcat sabote les relations hétérosexuelles", 'auteur': 'Chollet,  Mona'}, {'rang': 13, 'type_de_document': 'Bande dessinée adulte', 'reservations': 521.0, 'titre': 'Le jeune acteur', 'auteur': 'Sattouf,  Riad'}, {'rang': 14, 'type_de_document': 'Bande dessinée adulte', 'reservations': 510.0, 'titre': 'Une jeunesse au Moyen-Orient, 1992-1994', 'auteur': 'Sattouf,  Riad'}, {'rang': 37, 'type_de_document': 'DVD tous publics', 'reservations': 337.0, 'titre': 'Dune', 'auteur': None}, {'rang': 58, 'type_de_document': 'DVD tous publics', 'reservations': 266.0, 'titre': 'Onoda. 10 000 nuits dans la jungle', 'auteur': None}, {'rang': 73, 'type_de_document': 'Nouveauté', 'reservations': 241.0, 'titre': 'Chien 51 : roman', 'auteur': 'Gaudé,  Laurent'}, {'rang': 87, 'type_de_document': 'Livre adulte', 'reservations': 225.0, 'ti

Création du dataframe.
On remarque qu'il y a parfois des catégories qui ne donnent pas assez d'information sur le type d'ouvrage, à l'image de "Nouveauté", ou encore de "None". Nous pensons qu'il peut être pertinent d'étudier l'impact différé selon le genre d'ouvrage de l'exposition sur Youtube et , dans la limite du possible?

In [4]:
import pandas as pd
df_bibli= pd.json_normalize(wb['results'])
df_bibli.head(12)

,rang,type_de_document,reservations,titre,auteur
0,12,Livre adulte,522.0,Réinventer l'amour : comment le patriarcat sab...,"Chollet, Mona"
1,13,Bande dessinée adulte,521.0,Le jeune acteur,"Sattouf, Riad"
2,14,Bande dessinée adulte,510.0,"Une jeunesse au Moyen-Orient, 1992-1994","Sattouf, Riad"
3,37,DVD tous publics,337.0,Dune,None
4,58,DVD tous publics,266.0,Onoda. 10 000 nuits dans la jungle,None
5,73,Nouveauté,241.0,Chien 51 : roman,"Gaudé, Laurent"
6,87,Livre adulte,225.0,Le gosse : roman,"Olmi, Véronique"
7,109,Bande dessinée jeunesse,196.0,Attaque au clair de lune,"Oda, Eiichiro"
8,125,DVD nouveautés tous publics,177.0,West Side Story,None
9,136,None,170.0,Etés anglais,"Howard, Elizabeth Jane"
